In [1]:
from pathlib import Path
import subprocess

REPO_URL = "https://github.com/seoultechpse/fenicsx-colab.git"
ROOT = Path("/content")
REPO_DIR = ROOT / "fenicsx-colab"

subprocess.run(
  ["git", "clone", REPO_URL, str(REPO_DIR)],
  check=True
)

USE_CLEAN = False  # <--- Set True to remove existing environment
opts = "--clean" if USE_CLEAN else ""

get_ipython().run_line_magic(
    "run", f"{REPO_DIR / 'setup_fenicsx.py'} {opts}"
)

⚠️ Google Drive not mounted — using local cache (/content)
🔧 Installing FEniCSx environment...
✨ Loading FEniCSx Jupyter magic... %%fenicsx registered


---

In [2]:
%%fenicsx

import gmsh
from mpi4py import MPI
from dolfinx.io import XDMFFile
from dolfinx.io import gmsh as gmshio

# Initialize Gmsh
gmsh.initialize()

# Create a new Gmsh model
model = gmsh.model()

# Create a sphere centered at origin (0,0,0) with radius 1
sphere = model.occ.addSphere(0, 0, 0, 1, tag=1)

# Synchronize CAD representation with Gmsh model
model.occ.synchronize()

# Add sphere to 3D physical group (required for mesh export)
model.add_physical_group(dim=3, tags=[sphere])

# Generate 3D tetrahedral mesh
model.mesh.generate(dim=3)

# Convert Gmsh model to DOLFINx mesh
# Mesh is created on rank 0 and distributed across all MPI processes
# Returns MeshData named tuple with mesh, cell_tags, facet_tags, etc.
mesh_data = gmshio.model_to_mesh(model, MPI.COMM_WORLD, rank=0)

# Extract mesh components from MeshData
mesh = mesh_data.mesh
cell_tags = mesh_data.cell_tags
facet_tags = mesh_data.facet_tags

# Finalize Gmsh (no longer needed)
gmsh.finalize()

# Save mesh as XDMF file for use in ParaView
with XDMFFile(MPI.COMM_WORLD, "sphere_mesh.xdmf", "w") as xdmf:
    # Write mesh
    xdmf.write_mesh(mesh)

    # Save cell tags if available (useful for material or region identification)
    if cell_tags is not None:
        xdmf.write_meshtags(cell_tags, mesh.geometry)

    # Save facet tags if available (useful for boundary condition application)
    if facet_tags is not None:
        xdmf.write_meshtags(facet_tags, mesh.geometry)

print(f"Mesh saved successfully: sphere_mesh.xdmf")
print(f"Total number of cells: {mesh.topology.index_map(3).size_local}")
print(f"Total number of vertices: {mesh.topology.index_map(0).size_local}")

Info    : Meshing 1D...
Info    : [ 40%] Meshing curve 2 (Circle)
Info    : Done meshing 1D (Wall 0.000211791s, CPU 0.000161s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Sphere, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0143092s, CPU 0.012345s)
Info    : Meshing 3D...
Info    : 3D Meshing 1 volume with 1 connected component
Info    : Tetrahedrizing 160 nodes...
Info    : Done tetrahedrizing 168 nodes (Wall 0.00234786s, CPU 0.002347s)
Info    : Reconstructing mesh...
Info    :  - Creating surface mesh
Info    :  - Identifying boundary edges
Info    :  - Recovering boundary
Info    : Done reconstructing mesh (Wall 0.00508576s, CPU 0.004052s)
Info    : Found volume 1
Info    : It. 0 - 0 nodes created - worst tet radius 3.49551 (nodes removed 0 0)
Info    : 3D refinement terminated (204 nodes total):
Info    :  - 0 Delaunay cavities modified for star shapeness
Info    :  - 0 nodes could not be inserted
Info    :  - 698 tetrahedra created in 0.00329444 sec. (211871 tets/

In [3]:
from google.colab import files
files.download('sphere_mesh.xdmf')
files.download('sphere_mesh.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>